In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import *
import numpy as np
import copy
from functools import partial
from tfrecord_lite import decode_example
import sys, os
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *
NUMERIC_COLUMS = ['x1', 'y1', 'w', 'h', 'blur']
CATEGORICAL_COLUMS = ['blur', 'expression', 'illumniation',  'invalid', 
'occlusion', 'pose']


In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
# feature_description = {
#     'filename': tf.io.VarLenFeature(tf.string),
#     'shape' : tf.io.FixedLenFeature([3], tf.int64),
#     'image' : tf.io.FixedLenFeature([], tf.string),
#     'bbxs' : tf.io.VarLenFeature(tf.int64), 
#     'faces' : tf.io.FixedLenFeature([1], tf.int64),
# }


# def parse_function(example_proto):
#   # Parse the input `tf.Example` proto using the dictionary above.
#   example = tf.io.parse_single_example(example_proto, feature_description)
#   image, _ = decode_img(example['image'])
#   image = tf.cast(image, tf.uint8)
#   #   print(example['bbxs'])
#   bbxs = tf.cast(example['bbxs'], tf.int64)

#   if isinstance(bbxs, tf.SparseTensor):
#       bbxs = tf.sparse.to_dense(bbxs)
      
#   return image, bbxs


In [ ]:
train_filenames = [OUTPUT_TRAIN_TFRECORD]
train_dataset = prepare_dataset(train_filenames)

In [ ]:
validation_filenames = [OUTPUT_VALIDATION_TFRECORD]
validation_dataset = prepare_dataset(validation_filenames)

In [ ]:
base_model = applications.VGG19(
    include_top=False,
    input_shape=(*IMAGE_SIZES, 3),
    weights='imagenet'
)
base_model.trainable = False

In [ ]:
model = Sequential([base_model])

model.add(layers.Conv2D(1024, (4, 4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(1024, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(1024, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(512, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(512, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(256, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(128, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
model.add(layers.Conv2D(64, (4,4), activation='relu', padding="same"))
model.add(layers.MaxPool2D(strides=(2, 2), padding="same"))
# model.add(layers.Conv2D(512, (3,3), activation='relu', padding="same"))
# model.add(layers.MaxPool2D(strides=(2, 2)))
# model.add(layers.Conv2D(256, (3,3), activation='relu', input_shape=(1, 1, 256), data_format='channels_first'))
# model.add(layers.MaxPool2D((2, 2)))
# model.add(layers.Conv2D(512, (3,3), activation='relu'))
# model.add(layers.MaxPool2D((2, 2)))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
# # 10 because its amount of classes we have
model.add(layers.Dense(MAX_BBXS * 4, activation='relu'))

In [ ]:
model.save(RAW_MODEL)

In [ ]:
model.summary()

In [ ]:
gl = tfa.losses.GIoULoss()

In [ ]:
def loss_fn_practical(y_true, y_pred):
        # now contains 32 lists (a batch) of bbxs -> shape is (32, 7876)
        # print(y_true)
        batch_size = y_pred.shape[0]
        bbx_true = y_true

        # now contains 32 lists (a batch) of bbxs here we have to double access [0] in order to get the entry itself 
        # -> shape is (32, 1, 1, 7876)
        bbx_pred = tf.reshape(y_pred, (batch_size, MAX_BBXS * 4))

        losses = []
        curr_true = []
        curr_pred = []

        unpacked_true = tf.unstack(bbx_true)
        unpacked_pred = tf.unstack(bbx_pred)
        for i in range(batch_size):
            curr_true = unpacked_true[i] 
            curr_pred = unpacked_pred[i]


            curr_true = tf.split(curr_true, MAX_BBXS)
            curr_pred = tf.split(curr_pred, MAX_BBXS)

            if len(curr_true) == 0:
                curr_true.append(tf.concat([0., 0.,0.,0.]))

            curr_loss = gl(curr_true, curr_pred)

            losses.append(curr_loss)
# tf.math.reduce_mean(losses, axis=-1)
        return tf.math.real(losses)

In [ ]:
def loss_fn(y_true, y_pred):
        # now contains 32 lists (a batch) of bbxs -> shape is (32, 7876)
        bbx_true = y_true

        # now contains 32 lists (a batch) of bbxs here we have to double access [0] in order to get the entry itself 
        # -> shape is (32, 1, 1, 7876)
        bbx_pred = tf.reshape(y_pred, (1, 7876))

        losses = []
        curr_true = []
        curr_pred = []

        unpacked_true = tf.unstack(bbx_true)
        unpacked_pred = tf.unstack(bbx_pred)
        for i in range(1):
            curr_true = unpacked_true[i] 
            curr_pred = unpacked_pred[i]


            curr_true = tf.split(curr_true, 1969)
            curr_pred = tf.split(curr_pred, 1969)

            if len(curr_true) == 0:
                curr_true.append(tf.concat([0., 0.,0.,0.]))

            curr_loss = gl(curr_true, curr_pred)

            losses.append(curr_loss)

        return tf.math.reduce_mean(losses, axis=-1)

In [ ]:
for record in tf.compat.v1.python_io.tf_record_iterator(OUTPUT_TRAIN_TFRECORD):
    img, bbxs = parse_function(record)
    bbxs = tf.reshape(bbxs, (1, 7876))
    bbxs2 = copy.copy(bbxs)
    bbxs2 = tf.reshape(bbxs2, (1, 1, 1, 7876))
    x = loss_fn(bbxs, bbxs2)
    print(x)
    break

In [ ]:
lr = 1e-3
opt = optimizers.Ftrl(learning_rate=1.0)
# optimizer: Ftrl, learning_rate: 1.0, loss_fn: mean_squared_logarithmic_error, accuracy: [0.9937806129455566, 0.9937731027603149, 0.9937837719917297, 0.9937859773635864, 0.9937738180160522], loss: [0.07335256040096283, 0.07296431064605713, 0.07259127497673035, 0.07223349809646606, 0.07189014554023743], 
model.compile(optimizers.SGD(), loss=loss_fn_practical, metrics='accuracy', run_eagerly=True) #run_eagerly=True

In [ ]:
# next(train_dataset.as_numpy_iterator())

In [ ]:
history = model.fit(train_dataset, epochs=6)
# history = model.fit(train_dataset, epochs=6, validation_data=validation_dataset)

In [ ]:
# history['accuracy']
model.history.history['loss'] 

In [ ]:
OUTPUT_MODEL

In [ ]:
model.save(os.path.join(MODELS_PATH, "face_recog_20211204_1323.h5"))